In [2]:
# This is only needed if the notebook is run in VSCode
from IPython.display import clear_output, DisplayHandle
def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [3]:
import sys
sys.path.append('..')
import sklearn
from tsai.basics import *
from swdf.utils import *
my_setup(sklearn)
from matplotlib import dates as mdates
import wandb
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from itertools import chain
import more_itertools as mit

os              : Linux-6.8.0-40-generic-x86_64-with-glibc2.31
python          : 3.10.12
tsai            : 0.3.10
fastai          : 2.7.15
fastcore        : 1.5.49
sklearn         : 1.5.1
torch           : 2.2.2+cu121
device          : 1 gpu (['NVIDIA GeForce RTX 3070 Ti Laptop GPU'])
cpu cores       : 14
threads per cpu : 1
RAM             : 15.28 GB
GPU memory      : [8.0] GB


In [4]:
# This is only needed for the MIT supercloud, to fix fastai's LRFinder error 
if torch.cuda.is_available() and torch.cuda.device_count() == 0:
    from fastai.callback.schedule import LRFinder

    @patch_to(LRFinder)
    def after_fit(self):
        self.learn.opt.zero_grad() # Needed before detaching the optimizer for future fits
        tmp_f = self.path/self.model_dir/self.tmp_p/'_tmp.pth'
        if tmp_f.exists():
            self.learn.load(f'{self.tmp_p}/_tmp', with_opt=True, device='cpu')
            self.tmp_d.cleanup()

# Forecasting solar drivers F10, S10, M10 and Y10

Some hints about hyperparameters:
- According to the authors of PatchTST: "The ideal patch length may depend on the dataset, 
but P between {8, 16} seems to be general good numbers."

In [5]:
config_base = yaml2dict('./config/base.yaml', attrdict=True)
config_solfsmy = yaml2dict('./config/solfsmy.yaml', attrdict=True)
config_train = config_solfsmy.train
config_data = config_solfsmy.data 

# Merge the two configs (the second one overrides the first one for any keys that are present in both)
config = AttrDict({**config_base,
                   **config_train, 
                   "data":AttrDict({**config_data})})

# Add the architecture config
if config.arch_name.lower() == 'patchtst':
    config.arch = yaml2dict('./config/patchtst.yaml', attrdict=True)
else:
    config.arch = AttrDict()

In [6]:
run = wandb.init(project=config.wandb.project, 
                 config=config,
                 group=config.wandb.group,
                 mode=config.wandb.mode, 
                 anonymous='never') if config.wandb.enabled else None
config = dict2attrdict(run.config) if config.wandb.enabled else config

In [7]:
config

```json
{ 'arch': {'attn_dropout': 0.4, 'd_model': 1024, 'dropout': 0.2, 'n_heads': 32},
  'arch_config_file': 'config/patchtst.yaml',
  'arch_name': 'PatchTST',
  'bs': 32,
  'data': { 'add_time_channels': False,
            'data_columns': ['F10', 'S10', 'M10', 'Y10'],
            'data_nb': './solfsmy_data.ipynb',
            'data_path': '../data/SOLFSMY.TXT',
            'data_time_columns': ['Year', 'DDD'],
            'data_url': 'https://sol.spacenvironment.net/JB2008/indices/SOLFSMY.TXT',
            'df_save_path': './preprocessed_data/dataframes/solfsmy.pkl',
            'force_download': False,
            'preproc_pipe_save_path': './preprocessed_data/pipelines/preproc_solfsmy.pkl'},
  'deltaHL': 4.0,
  'horizon': 6,
  'init_weights': False,
  'is_optuna_study': False,
  'lookback': 24,
  'lr_max': None,
  'n_epoch': 300,
  'partial_n': 0.1,
  'seed': 42,
  'test_end_datetime': '2018-12-31 12:00:00',
  'test_start_datetime': '2012-10-01 12:00:00',
  'valid_start_datetime': '2018-01-01 12:00:00',
  'wandb': { 'enabled': False,
             'group': None,
             'log_learner': True,
             'mode': 'offline',
             'project': 'swdf'}}
```

In [8]:
df, preproc_pipe = generate_preprocessed_data(config.data)

df.head()

,Year,DDD,JulianDay,F10,F81c,S10,S81c,M10,M81c,Y10,Y81c,Ssrc,Ssrc_F10,Ssrc_S10,Ssrc_M10,Ssrc_Y10,Date
0,1997,1,2450450.0,72.400002,78.000000,74.000000,79.199997,65.400002,73.800003,61.900002,70.699997,1B11,1,B,1,1,1997-01-01 12:00:00
1,1997,2,2450451.0,72.099998,77.900002,73.800003,79.099998,66.900002,73.699997,63.400002,70.400002,1B11,1,B,1,1,1997-01-02 12:00:00
2,1997,3,2450452.0,73.300003,77.599998,75.000000,79.099998,70.099998,73.599998,64.900002,70.199997,1B11,1,B,1,1,1997-01-03 12:00:00
3,1997,4,2450453.0,73.800003,77.300003,76.300003,79.000000,71.599998,73.400002,65.500000,69.800003,1B11,1,B,1,1,1997-01-04 12:00:00
4,1997,5,2450454.0,74.400002,76.900002,77.599998,78.900002,72.400002,73.300003,66.699997,69.599998,1B11,1,B,1,1,1997-01-05 12:00:00


## Train

In the paper by [Licata et al. (2020)]((https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2020SW002496)), authors use a period from October 2012 through the end of 2018 for the benchmarking. Therefore, we will set the test set as the same period for our analysis, using the column Date as the timestamp, from October 2012 to the end of 2018. Everything before the test set will be used for training, and everything after the test set will be used for validation. In this paper, the authors also present some the thresholds for categorizing solar activity levels.

In [9]:
test_start_datetime = config.test_start_datetime
test_end_datetime = config.test_end_datetime

thresholds = yaml2dict("distributions/thresholds.yaml")
solact_levels = yaml2dict("distributions/thresholds_labels.yaml")

{'F10': [(0, 75), (76, 150), (151, 190), (191, 99999)], 'S10': [(0, 65), (66, 150), (151, 215), (216, 99999)], 'M10': [(0, 72), (73, 144), (145, 167), (168, 99999)], 'Y10': [(0, 81), (82, 148), (149, 165), (166, 99999)]}


In [10]:
df_F10 = load_object("./preprocessed_data/dataframes/F10_historical.pkl")

df_F107_cat = get_classified_columns(df_F10, thresholds, activity_levels={'F10': solact_levels['F10']})['F10_Cat']
historical_distribution = df_F107_cat.value_counts(normalize=True).to_dict()

historical_distribution

{'moderate': 0.5065749000307598,
 'low': 0.18390495232236234,
 'high': 0.16145032297754536,
 'elevated': 0.14806982466933252}

In [11]:
test_period = (df.Date >= test_start_datetime) & (df.Date <= test_end_datetime)

df_cat = get_classified_columns(df, thresholds, solact_levels)
df_cat = df_cat[~(test_period)]

# Function parameter calculation
test_size = df[test_period].shape[0] / df.shape[0]
train_val_size = 1 - test_size
val_size = 0.15 / train_val_size
print(f"Test size: {test_size}, Validation size: {val_size}, Train size: {1 - test_size - 0.15}")

# Best segment size found: 250, val_size: 0.15, test_size: 0.65
best_comb, segments, distribution = find_closest_distribution(df_cat['F10_Cat'], historical_distribution, 250, val_size) 

Test size: 0.22903290529695025, Validation size: 0.19456083279115158, Train size: 0.6209670947030498
Total number of segments:30, Number of segments for validation: 5 (16.67%)


142506it [00:02, 70424.57it/s]

The closest group of segments to F10.7 categories has an euclidean distance of 0.005517135807631744


In [13]:
best_comb_idxs = [segments[i] for i in best_comb]
validation_segments = (df.index.isin(chain.from_iterable(best_comb_idxs)))

train_df = df[~validation_segments & ~test_period] 
train_distribution = get_classified_columns(train_df, thresholds, solact_levels)['F10_Cat'].value_counts(normalize=True).to_dict()

train_distribution

{'moderate': 0.5067145534041224,
 'low': 0.28856964397251716,
 'elevated': 0.1316364772017489,
 'high': 0.0730793254216115}

 ### Apply a sliding window. 

In [17]:
a = np.arange(0,len(segments))
train_comb = list(np.setdiff1d(a, best_comb))
data_columns = config.data.data_columns

X_val, y_val, split_val = sliding_window_generator(df, 0, data_columns=data_columns, config=config, comb=best_comb, segments=segments)
X_train, y_train, split_train = sliding_window_generator(df, split_val[-1]+1, data_columns=data_columns, config=config, comb=train_comb, segments=segments)
X_test, y_test, split_test = sliding_window_generator(df[test_period], split_train[-1]+1, data_columns=data_columns, config=config) 

X = np.concatenate([X_val, X_train, X_test])
y = np.concatenate([y_val, y_train, y_test])

splits = (split_train, split_val, split_test)
splits, X.shape, y.shape

(((#6230) [1136,1137,1138,1139,1140,1141,1142,1143,1144,1145...],
  (#1136) [0,1,2,3,4,5,6,7,8,9...],
  (#2254) [7366,7367,7368,7369,7370,7371,7372,7373,7374,7375...]),
 (9620, 4, 24),
 (9620, 4, 6))

In [18]:
# Now that we have defined the splits for this particular experiment, we'll scale
# the data
exp_pipe = sklearn.pipeline.Pipeline([
    ('scaler', TSStandardScaler(columns=data_columns)),
], verbose=True)

df_scaled = exp_pipe.fit_transform(df.copy())

save_object(exp_pipe, 'tmp/pipes/exp_pipe.pkl')
df_scaled.head()

[Pipeline] ............ (step 1 of 1) Processing scaler, total=   0.0s
tmp/pipes directory already exists.
Pipeline saved as tmp/pipes/exp_pipe.pkl


,Year,DDD,JulianDay,F10,F81c,S10,S81c,M10,M81c,Y10,Y81c,Ssrc,Ssrc_F10,Ssrc_S10,Ssrc_M10,Ssrc_Y10,Date
0,1997,1,2450450.0,-0.905757,78.000000,-0.761630,79.199997,-1.113339,73.800003,-1.293490,70.699997,1B11,1,B,1,1,1997-01-01 12:00:00
1,1997,2,2450451.0,-0.912979,77.900002,-0.766386,79.099998,-1.075890,73.699997,-1.254068,70.400002,1B11,1,B,1,1,1997-01-02 12:00:00
2,1997,3,2450452.0,-0.884091,77.599998,-0.737850,79.099998,-0.996001,73.599998,-1.214647,70.199997,1B11,1,B,1,1,1997-01-03 12:00:00
3,1997,4,2450453.0,-0.872054,77.300003,-0.706937,79.000000,-0.958552,73.400002,-1.198879,69.800003,1B11,1,B,1,1,1997-01-04 12:00:00
4,1997,5,2450454.0,-0.857611,76.900002,-0.676023,78.900002,-0.938580,73.300003,-1.167341,69.599998,1B11,1,B,1,1,1997-01-05 12:00:00


### Train

In [19]:
weights = {k: np.abs(train_distribution.get(k, 0) - historical_distribution.get(k, 0)) for k in set(train_distribution) | set(historical_distribution)}
factor = 1.0/sum(weights.values())
weights_raw = {k: v*factor for k, v in weights.items()}

weights = {'All': [weights_raw[k] for k in solact_levels['F10']]}
weights, weights_raw

({'All': [0.49933374244488,
   0.0006662575551199645,
   0.07840012484165651,
   0.4215998751583436]},
 {'elevated': 0.07840012484165651,
  'moderate': 0.0006662575551199645,
  'low': 0.49933374244488,
  'high': 0.4215998751583436})

In [ ]:
from swdf.losses import LossFactory
from swdf.metrics import ValidationMetricsHandler

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss = LossFactory(thresholds=thresholds, weights=weights).create(config.loss_func, **config.loss_config).to(device)

metrics_handler = load_object(config.metrics_handler_path)
metrics = metrics_handler.get_metrics()

wandb_callback = WandbCallback(log_preds=False)
cbs = L(wandb_callback) if config.wandb.enabled else L()
learn = TSForecaster(X, y, splits=splits, batch_size=config.bs,
                     pipelines=[preproc_pipe, exp_pipe], 
                     arch=config.arch_name, 
                     metrics=metrics,
                     loss_func=loss,
                     arch_config=dict(config.arch), 
                     init=config.init_weights,
                     cbs= cbs + ShowGraphCallback(), 
                     partial_n=config.partial_n)
learn.to(device)

try:
    lr_max = learn.lr_find().valley if config.lr_max is None else config.lr_max
except:
    lr_max = 1e-3

learn.fit_one_cycle(n_epoch=config.n_epoch, lr_max=config.lr_max)


In [243]:
learn.dls.loaders += [learn.dls.valid.new_dl(X[splits[2]], y[splits[2]])] # Add test datalaoder

# Remove the wandb callback to avoid errors when downloading the learner
if config.wandb.enabled:
    learn.remove_cb(wandb_callback)

In [244]:
valid_metrics = learn.recorder.metrics
%store valid_metrics

best_valid_metrics = None
%store -r best_valid_metrics

save_all_or_best = (
                    config.is_optuna_study and 
                    metrics_handler.are_best_values(config.main_metric, best_valid_metrics, valid_metrics)
                ) or not config.is_optuna_study


if save_all_or_best:
    best_valid_metrics = valid_metrics
    %store best_valid_metrics                                 

    # Save locally and in wandb if online and enabled
    learn.save_all(path='tmp', verbose=True) 
    if run is not None and config.wandb_mode and config.wandb_log_learner:
        # Save the learner (all tmp/dls, tmp/model.pth, and tmp/learner.pkl). 
        run.log_artifact('tmp', type='learner', name='solfsmy')
    
    # Save the best model
    model = {}
    model['models'] = [learn.model]
    model['lookback'] = config.lookback
    model['horizon'] = config.horizon
    model['data_columns'] = data_columns
    model['thresholds'] = thresholds

    torch.save(model, 'models/best/best_model_solfsmy.pth')

Stored 'best_valid_loss' (float)
Learner saved:
path          = 'tmp'
dls_fname     = '['dls_0.pth', 'dls_1.pth', 'dls_2.pth', 'dls_3.pth']'
model_fname   = 'model.pth'
learner_fname = 'learner.pkl'


In [245]:
# Log the test loss to wandb
test_loss = learn.validate(ds_idx=2)[0]

if run is not None:
    run.log(dict(test_loss=test_loss))

36.21273422241211


In [246]:
if run is not None:
    run.finish()